<a href="https://colab.research.google.com/github/AlekSSSandraJ/readnew/blob/main/NLP_CW_Task2_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖇 Deduplicating Training Data Makes Language Models

based on the code to deduplicate language model datasets as descrbed in the paper "Deduplicating Training Data Makes Language Models Better" by Katherine Lee, Daphne Ippolito, Andrew Nystrom, Chiyuan Zhang, Douglas Eck, Chris Callison-Burch and Nicholas Carlini.

# 1. Design and implement a text similarity detection system

The goal of this section is detecting plagiarism by measuring similarity between student answers and Wikipedia content, then automatically classifying the similarity level into:

* Cut – Near-identical matches (High similarity)
* Heavy – Substantial content matches (Significant similarity)
* Light – Moderate matches (Minimal similarity)
* Non – Below-defined similarity thresholds (No plagiarism detected)

In [34]:
import pandas as pd
import os
import glob
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!ls "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"
Data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"

answers  metadata.csv  ReadMe.md  wikipedia


In [36]:
def styled_df(df, title="Dataset"):
    print(f"\n🔹 {title}")
    #return df.head(10).style.set_table_styles(
    return df.style.set_table_styles(
        [{'selector': 'th',
          'props': [('background-color', '#add8e6'),  # Light blue header
                    ('color', 'black'),
                    ('font-weight', 'bold'),
                    ('text-align', 'center'),
                    ('font-family', 'Arial'),
                    ('border', '1.5px solid black')]},
         {'selector': 'td',
          'props': [('background-color', '#f4f4f4'),  # Light gray background
                    ('color', 'black'),
                    ('text-align', 'left'),
                    ('font-family', 'Arial'),
                    ('border', '1px solid lightgrey')]}]
    ).set_properties(**{'border': '1.5px solid lightgrey'})


In [37]:
def load_texts_from_directory(directory):
    text_data = []
    for file_path in glob.glob(os.path.join(directory, "*.txt")):
        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                text_data.append({"Filename": os.path.basename(file_path), "Content": file.read()})
        except UnicodeDecodeError:
            print(f"Skipping file {file_path} due to encoding issues.")

    # Convert list of dictionaries into a DataFrame
    return pd.DataFrame(text_data)

In [38]:
Data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/plagiarism/"
WIKI_PATH = os.path.join(Data_path, "wikipedia/")
ANSWERS_PATH = os.path.join(Data_path, "answers/")

# Load Wikipedia articles and student answers
wiki_df = load_texts_from_directory(WIKI_PATH)
answers_df = load_texts_from_directory(ANSWERS_PATH)

# 🔍  Underdating Datasets

In [39]:
!pip install datasketch

import pandas as pd
import os
import glob
import re
from datasketch import MinHash, MinHashLSH
from sklearn.metrics import jaccard_score


# Load Wikipedia articles and student answers
wiki_df = load_texts_from_directory(WIKI_PATH)
answers_df = load_texts_from_directory(ANSWERS_PATH)

print("Wikipedia Articles:")
display(styled_df(wiki_df))

print("\nStudent Answers:")
display(styled_df(answers_df))




Wikipedia Articles:

🔹 Dataset


,Filename,Content
0,orig_taskc.txt,"Vector space model (or term vector model) is an algebraic model for representing text documents (and any objects, in general) as vectors of identifiers, such as, for example, index terms. It is used in information filtering, information retrieval, indexing and relevancy rankings. Its first use was in the SMART Information Retrieval System. A document is represented as a vector. Each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. One of the best known schemes is tf-idf weighting (see the example below). The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). The vector space model has the following limitations: 1. Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality) 2. Search keywords must precisely match document terms; word substrings might result in a ""false positive match"" 3. Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a ""false negative match"". 4. The order in which the terms appear in the document is lost in the vector space representation."
1,orig_taske.txt,"In mathematics and computer science, dynamic programming is a method of solving problems that exhibit the properties of overlapping subproblems and optimal substructure (described below). The method takes much less time than naive methods. The term was originally used in the 1940s by Richard Bellman to describe the process of solving problems where one needs to find the best decisions one after another. By 1953, he had refined this to the modern meaning. The field was founded as a systems analysis and engineering topic that is recognized by the IEEE. Bellman's contribution is remembered in the name of the Bellman equation, a central result of dynamic programming which restates an optimization problem in recursive form. The word ""programming"" in ""dynamic programming"" has no particular connection to computer programming at all, and instead comes from the term ""mathematical programming"", a synonym for optimization. Thus, the ""program"" is the optimal plan for action that is produced. For instance, a finalized schedule of events at an exhibition is sometimes called a program. Programming, in this sense, means finding an acceptable plan of action, an algorithm. Optimal substructure means that optimal solutions of subproblems can be used to find the optimal solutions of the overall problem. For example, the shortest path to a goal from a vertex in a graph can be found by first computing the shortest path to the goal from all adjacent vertices, and then using this to pick the best overall path, as shown in Figure 1. In general, we can solve a problem with optimal substructure using a three-step process: 1. Break the problem into smaller subproblems. 2. Solve these problems optimally using this three-step process recursively. 3. Use these optimal solutions to construct an optimal solution for the original problem. The subproblems are, themselves, solved by dividing them into sub-subproblems, and so on, until we reach some simple case that is solvable in constant time. Figure 2. The subproblem graph for the Fibonacci sequence. That it is not a tree but a DAG indicates overlapping subproblems. To say that a problem has overlapping subproblems is to say that the same subproblems are used to solve many different larger problems. For example, in the Fibonacci sequence, F3 = F1 + F2 and F4 = F2 + F3 — computing each number involves computing F2. Because both F3 and F4 are needed to compute F5, a nai


Student Answers:

🔹 Dataset


,Filename,Content
0,g4pE_taskd.txt,"""Bayes' Theorem"" or ""Bayes' Rule"", or something called Bayesian reasoning The Bayesian Conspiracy is a multinational, interdisciplinary, and shadowy group of scientists that controls publication, grants, tenure, and the illicit traffic in grad students. The best way to be accepted into the Bayesian Conspiracy is to join the Campus Crusade for Bayes in high school or college, and gradually work your way up to the inner circles. . Bayes' Theorem Let and be sets. Conditional probability requires that (1) where denotes intersection (""and""), and also that (2) Therefore, (3) Now, let (4) so is an event in and for , then (5) (6) But this can be written (7) so This paper proposes a new measure called scaled inverse document frequency (SIDF) which evaluates the conditional specificity of query terms over a subset S of D and without making any assumption about term independence. S can be estimated from search results, OR searches, or computed from inverted index data. We have evaluated SIDF values from commercial search engines by submitting queries relevant to the financial investment domain. Results compare favorably across search engines and queries. Our approach has practical applications for `real-world scenarios like in Web Mining, Homeland Security, and keyword-driven marketing research scenarios."
1,g0pB_taskc.txt,"Vector space model is an algebraic model for representing text documents (and in general, any objects) as vectors of identifiers, such as, for example, index terms. Its first use was in the SMART Information Retrieval System. It is used in information filtering, information retrieval, indexing and relevancy rankings. A document is represented as a vector, and each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). One of the best known schemes is tf-idf weighting, proposed by Salton, Wong and Yang. In the classic vector space model, the term specific weights in the document vectors are products of local and global parameters. Relevancy rankings of documents in a keyword search can be calculated, using the assumptions of document similarities theory, by comparing the deviation of angles between each document vector and the original query vector where the query is represented as same kind of vector as the documents. The vector space model has the following limitations: * Search keywords must precisely match document terms; word substrings might result in a ""false positive match""; * Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a ""false negative match""; * The order in which the terms appear in the document is lost in the vector space representation; * Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality)."
2,g0pE_taskd.txt,"Bayes Theorem is an important theorem relating conditional probabilities, it allows us to calculate PROB(A|B) from PROB(B|A). Bayes Theorem is important because it can save us from gathering vast amounts of statistical evidence. The main theory is PROB(A|B) = PROB(B|A) * PROB(A) /PROB(B), it means Using PROB(WIN|RAIN) from earlier, we can find the probability that it rained on a day that Harry won a race."
3,g3pB_taskd.txt,"Bayes' theorem (often called Bayes' law) connects the conditional and marginal probabilities of two arbitrary events. One of its uses is calculating posterior probabilities given observations. Bayes' theorem plays a key role in the debate around the pr

# ⏳ PRE PROCESSING


Effective preprocessing ensures better accuracy in similarity detection and helps eliminate irrelevant variations that could lead to false positives or negatives. Before I implement a text similarity detection system using MinHash signatures, I will preprocess both wkipedia and the students answers.

MinHash is designed to capture similarity, and altering word forms by using lemmatization or stemming can reduce the effectiveness of exact matching since it changes the original substrings. As the main goal is to incorporate an automated classification framework to categorise detected similarities into four designated levels lemmatization and stemming won't be added to pre processing steps and I will focus instead on:

    - Converting words to lowercase
    - Removing punctuation and special characters
    - Tokenizes into words

In [40]:
import nltk
import os
import shutil

# 🔹 Delete any corrupted `punkt` files
shutil.rmtree("/root/nltk_data/tokenizers/punkt", ignore_errors=True)

# 🔹 Manually download and set the path
nltk.download('punkt')
nltk.data.path.append('/root/nltk_data')

print("✅ Successfully reinstalled punkt!")

✅ Successfully reinstalled punkt!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt_tab')


def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Standardize whitespace (replace multiple spaces with a single space)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)

    return text

# Apply preprocessing to both datasets
wiki_df["Processed_Content"] = wiki_df["Content"].apply(preprocess_text)
answers_df["Processed_Content"] = answers_df["Content"].apply(preprocess_text)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [42]:
# Before & After Processing

display(styled_df(wiki_df[["Filename", "Content", "Processed_Content"]], "Wikipedia Articles (Before & After Processing)"))



🔹 Wikipedia Articles (Before & After Processing)


,Filename,Content,Processed_Content
0,orig_taskc.txt,"Vector space model (or term vector model) is an algebraic model for representing text documents (and any objects, in general) as vectors of identifiers, such as, for example, index terms. It is used in information filtering, information retrieval, indexing and relevancy rankings. Its first use was in the SMART Information Retrieval System. A document is represented as a vector. Each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. One of the best known schemes is tf-idf weighting (see the example below). The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). The vector space model has the following limitations: 1. Long documents are poorly represented because they have poor similarity values (a small scalar product and a large dimensionality) 2. Search keywords must precisely match document terms; word substrings might result in a ""false positive match"" 3. Semantic sensitivity; documents with similar context but different term vocabulary won't be associated, resulting in a ""false negative match"". 4. The order in which the terms appear in the document is lost in the vector space representation.",vector space model or term vector model is an algebraic model for representing text documents and any objects in general as vectors of identifiers such as for example index terms it is used in information filtering information retrieval indexing and relevancy rankings its first use was in the smart information retrieval system a document is represented as a vector each dimension corresponds to a separate term if a term occurs in the document its value in the vector is nonzero several different ways of computing these values also known as term weights have been developed one of the best known schemes is tfidf weighting see the example below the definition of term depends on the application typically terms are single words keywords or longer phrases if the words are chosen to be the terms the dimensionality of the vector is the number of words in the vocabulary the number of distinct words occurring in the corpus the vector space model has the following limitations 1 long documents are poorly represented because they have poor similarity values a small scalar product and a large dimensionality 2 search keywords must precisely match document terms word substrings might result in a false positive match 3 semantic sensitivity documents with similar context but different term vocabulary wont be associated resulting in a false negative match 4 the order in which the terms appear in the document is lost in the vector space representation
1,orig_taske.txt,"In mathematics and computer science, dynamic programming is a method of solving problems that exhibit the properties of overlapping subproblems and optimal substructure (described below). The method takes much less time than naive methods. The term was originally used in the 1940s by Richard Bellman to describe the process of solving problems where one needs to find the best decisions one after another. By 1953, he had refined this to the modern meaning. The field was founded as a systems analysis and engineering topic that is recognized by the IEEE. Bellman's contribution is remembered in the name of the Bellman equation, a central result of dynamic programming which restates an optimization problem in recursive form. The word ""programming"" in ""dynamic programming"" has no particular connection to computer programming at all, and instead comes from the term ""mathematical programming"", a synonym for optimization. Thus, the ""program"" is the op

In [43]:
display(styled_df(answers_df[["Filename", "Content", "Processed_Content"]], "Student Answers (Before & After Processing)"))


🔹 Student Answers (Before & After Processing)


,Filename,Content,Processed_Content
0,g4pE_taskd.txt,"""Bayes' Theorem"" or ""Bayes' Rule"", or something called Bayesian reasoning The Bayesian Conspiracy is a multinational, interdisciplinary, and shadowy group of scientists that controls publication, grants, tenure, and the illicit traffic in grad students. The best way to be accepted into the Bayesian Conspiracy is to join the Campus Crusade for Bayes in high school or college, and gradually work your way up to the inner circles. . Bayes' Theorem Let and be sets. Conditional probability requires that (1) where denotes intersection (""and""), and also that (2) Therefore, (3) Now, let (4) so is an event in and for , then (5) (6) But this can be written (7) so This paper proposes a new measure called scaled inverse document frequency (SIDF) which evaluates the conditional specificity of query terms over a subset S of D and without making any assumption about term independence. S can be estimated from search results, OR searches, or computed from inverted index data. We have evaluated SIDF values from commercial search engines by submitting queries relevant to the financial investment domain. Results compare favorably across search engines and queries. Our approach has practical applications for `real-world scenarios like in Web Mining, Homeland Security, and keyword-driven marketing research scenarios.",bayes theorem or bayes rule or something called bayesian reasoning the bayesian conspiracy is a multinational interdisciplinary and shadowy group of scientists that controls publication grants tenure and the illicit traffic in grad students the best way to be accepted into the bayesian conspiracy is to join the campus crusade for bayes in high school or college and gradually work your way up to the inner circles bayes theorem let and be sets conditional probability requires that 1 where denotes intersection and and also that 2 therefore 3 now let 4 so is an event in and for then 5 6 but this can be written 7 so this paper proposes a new measure called scaled inverse document frequency sidf which evaluates the conditional specificity of query terms over a subset s of d and without making any assumption about term independence s can be estimated from search results or searches or computed from inverted index data we have evaluated sidf values from commercial search engines by submitting queries relevant to the financial investment domain results compare favorably across search engines and queries our approach has practical applications for realworld scenarios like in web mining homeland security and keyworddriven marketing research scenarios
1,g0pB_taskc.txt,"Vector space model is an algebraic model for representing text documents (and in general, any objects) as vectors of identifiers, such as, for example, index terms. Its first use was in the SMART Information Retrieval System. It is used in information filtering, information retrieval, indexing and relevancy rankings. A document is represented as a vector, and each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. Several different ways of computing these values, also known as (term) weights, have been developed. The definition of term depends on the application. Typically terms are single words, keywords, or longer phrases. If the words are chosen to be the terms, the dimensionality of the vector is the number of words in the vocabulary (the number of distinct words occurring in the corpus). One of the best known schemes is tf-idf weighting, proposed by Salton, Wong and Yang. In the classic vector space model, the term specific weights in the document vectors are products of local and global parameters. Relevancy rankings of documents in a keyword search can be calculated, using the assumptions of document similarities theory, by comparing the deviation of angles between each document vector and the original query vector where the query is represented as same ki

In order to design and implement a text similarity detection system using MinHash, you need to:

* Preprocess the text → Convert to lowercase, remove punctuation, and standardize whitespace.

* Generate n-gram shingles (for robust comparison).
* Compute MinHash signatures (for fast approximate similarity detection).
* Compare text similarities using Jaccard similarity.
* Categorize matches into Cut, Heavy, Light, Non based on predefined thresholds.


In [44]:
# Function to compute MinHash signature for a given text
def compute_minhash(text, num_perm=128):
    minhash = MinHash(num_perm=num_perm)
    words = text.split()
    for word in words:
        minhash.update(word.encode('utf8'))
    return minhash


In [ ]:

# Compute MinHash signatures for Wikipedia articles
wiki_df["MinHash"] = wiki_df["Processed_Content"].apply(lambda x: compute_minhash(x))

# Compute MinHash signatures for Student Answers
answers_df["MinHash"] = answers_df["Processed_Content"].apply(lambda x: compute_minhash(x))

# Initialize LSH for efficient approximate matching
lsh = MinHashLSH(threshold=0.1, num_perm=128)

# Add Wikipedia documents to LSH index
for i, row in wiki_df.iterrows():
    lsh.insert(f"wiki_{i}", row["MinHash"])

# Function to compute Jaccard similarity between two MinHash signatures
def compute_jaccard_similarity(minhash1, minhash2):
    return minhash1.jaccard(minhash2)

# Function to classify plagiarism level based on similarity score
def classify_similarity(similarity):
    if similarity >= 0.9:
        return "Cut"  # Near copy
    elif 0.7 <= similarity < 0.9:
        return "Light"  # Minor paraphrasing
    elif 0.4 <= similarity < 0.7:
        return "Heavy"  # Significant rewriting
    else:
        return "Non"  # Independent work

# Compare each student answer to Wikipedia articles
matches = []
for i, ans_row in answers_df.iterrows():
    best_match = None
    best_score = 0

    for j, wiki_row in wiki_df.iterrows():
        score = compute_jaccard_similarity(ans_row["MinHash"], wiki_row["MinHash"])

        if score > best_score:
            best_score = score
            best_match = wiki_row["Filename"]

    # Store the results
    matches.append({
        "Answer_Filename": ans_row["Filename"],
        "Best_Matching_Wikipedia": best_match,
        "Similarity_Score": best_score,
        "Category": classify_similarity(best_score)
    })

# Convert matches to a DataFrame
results_df = pd.DataFrame(matches)

# Display the classified plagiarism results
print("\nPlagiarism Detection Results:")
display(styled_df(results_df))

In [3]:
def styled_df(df):
    return df.style.set_table_styles(
        [{'selector': 'th',
          'props': [('background-color', '#add8e6'),  # Light blue header
                    ('color', 'black'),
                    ('font-weight', 'bold'),
                    ('text-align', 'center'),
                    ('font-family', 'Arial'),
                    ('border', '1.5px solid black')]},
         {'selector': 'td',
          'props': [('background-color', '#f4f4f4'),  # Light gray background
                    ('color', 'black'),
                    ('text-align', 'left'),
                    ('font-family', 'Arial'),
                    ('border', '1px solid lightgrey')]}]
    ).set_properties(**{'border': '1.5px solid lightgrey'})


In [6]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def preprocess_text(text, n_shingle=3):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Standardize whitespace (replace multiple spaces with a single space)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)

    # Generate n-gram shingles
    shingles = list(ngrams(tokens, n_shingle))
    shingle_strings = [" ".join(shingle) for shingle in shingles]

    return shingle_strings

